In [1]:
import pandas as pd
from keras.layers import Embedding, LSTM , Bidirectional
from keras.layers import Dense , Dropout
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import one_hot

C:\Users\Aishwarya\Anaconda3\lib\site-packages\requests\__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.4) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
Using TensorFlow backend.


In [2]:
df = pd.read_csv(r"C:\\Users\\Aishwarya\\Desktop\\sourabh\\Fake-News-Classifier-BiLSTM-master\\Fake-News-Classifier-BiLSTM-master\\train.csv")
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [3]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [4]:
df.shape

(20800, 5)

In [5]:
#Dropping the NAN Values
df = df.dropna()

In [6]:
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [7]:
#splitting of data into independent and dependent feauture
x = df.drop("label", axis = 1)
y = df["label"]

In [8]:
x.shape

(18285, 4)

In [9]:
y.shape

(18285,)

In [10]:
y.value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [11]:
#Reset index of our dataset as we drop the null values from our dataset
messages = x.copy()
messages.reset_index(inplace = True)

In [12]:
messages.shape

(18285, 5)

In [13]:
messages.columns

Index(['index', 'id', 'title', 'author', 'text'], dtype='object')

In [14]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [15]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Aishwarya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [16]:
#re.sub()
#Replace with regular expression
#Replace multiple substrings with the same string
#Replace using the matched part
#Get the count of replaced parts
#Here we are going to replace all regural expression which are not between a to z or A to Z with whitespace.

#result.lower:
#By this we are going to replace all the words in Lower case,So that all are treated as equally

#ps.stem()
#Stemming is the process of producing morphological variants of a root/base word. Stemming programs are
#commonly referred to as stemming algorithms or stemmers. A stemming algorithm reduces the words “chocolates”, “chocolatey”, “choco” to the root word, “chocolate” and “retrieval”, “retrieved”, “retrieves” reduce to the stem “retrieve”

In [17]:
ps = PorterStemmer()
corpus = []
for i in range(0 , len(messages)):
    review = re.sub("[^a-zA-Z]"," ", messages["title"][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words("english")]
    review = " ".join(review)
    corpus.append(review)

In [18]:
len(corpus)

18285

In [19]:
corpus[:15]

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time']

In [20]:
onehot_repr = [one_hot(words,5000) for words in corpus]
onehot_repr[:5]

[[4156, 1491, 633, 4426, 1651, 2218, 412, 3362, 2021, 712],
 [1064, 635, 1489, 3863, 353, 2167, 4932],
 [4999, 1670, 3858, 3448],
 [1640, 3061, 4204, 3256, 2614, 3586],
 [1596, 353, 4467, 4281, 2361, 2003, 353, 76, 4221, 4988]]

In [21]:
sent_length = 30
embeded_docs = pad_sequences(onehot_repr,padding= 'post',maxlen = sent_length)
embeded_docs

array([[4156, 1491,  633, ...,    0,    0,    0],
       [1064,  635, 1489, ...,    0,    0,    0],
       [4999, 1670, 3858, ...,    0,    0,    0],
       ...,
       [1219, 3820, 4745, ...,    0,    0,    0],
       [1202,  893, 4346, ...,    0,    0,    0],
       [2611, 4702,  972, ...,    0,    0,    0]])

In [22]:
embeded_docs[0]

array([4156, 1491,  633, 4426, 1651, 2218,  412, 3362, 2021,  712,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0])

In [23]:
import numpy as np

In [24]:
X = np.array(embeded_docs)
Y= np.array(y)

In [25]:
y.shape

(18285,)

In [26]:
from sklearn.model_selection import train_test_split
X_train , X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3 , random_state = 42)

In [27]:
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(5000, embedding_vector_features, input_length = sent_length))
model.add(LSTM(100))
model.add(Dense(1, activation = 'sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 30, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


In [28]:
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [29]:
model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 10,batch_size = 64)

Epoch 1/10
200/200 [==============================] - 5s 23ms/step - loss: 0.3511 - accuracy: 0.8203 - val_loss: 0.2077 - val_accuracy: 0.9143
Epoch 2/10
200/200 [==============================] - 4s 21ms/step - loss: 0.1404 - accuracy: 0.9469 - val_loss: 0.2398 - val_accuracy: 0.9172
Epoch 3/10
200/200 [==============================] - 4s 22ms/step - loss: 0.0989 - accuracy: 0.9666 - val_loss: 0.2419 - val_accuracy: 0.9158
Epoch 4/10
200/200 [==============================] - 4s 21ms/step - loss: 0.0749 - accuracy: 0.9745 - val_loss: 0.2558 - val_accuracy: 0.9147
Epoch 5/10
200/200 [==============================] - 4s 22ms/step - loss: 0.0565 - accuracy: 0.9819 - val_loss: 0.3258 - val_accuracy: 0.9131
Epoch 6/10
200/200 [==============================] - 5s 23ms/step - loss: 0.0484 - accuracy: 0.9853 - val_loss: 0.3836 - val_accuracy: 0.9079
Epoch 7/10
200/200 [==============================] - 5s 23ms/step - loss: 0.0371 - accuracy: 0.9894 - val_loss: 0.2976 - val_accuracy: 0.9069

In [35]:
y_pred = model.predict_classes(X_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [36]:
from sklearn.metrics import confusion_matrix

In [37]:
confusion_matrix(y_test,y_pred)

array([[2814,  293],
       [ 217, 2162]], dtype=int64)

In [38]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test , y_pred)

0.907036091870215

In [39]:
embedding_vector_features = 40
model1 = Sequential()
model1.add(Embedding(5000, embedding_vector_features, input_length = sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dense(1, activation = 'sigmoid'))
model1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 30, 40)            200000    
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               112800    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 201       
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________


In [40]:
model1.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [41]:
model1.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 10,batch_size = 64)

Epoch 1/10
200/200 [==============================] - 5s 27ms/step - loss: 0.3081 - accuracy: 0.8538 - val_loss: 0.1903 - val_accuracy: 0.9225
Epoch 2/10
200/200 [==============================] - 5s 24ms/step - loss: 0.1324 - accuracy: 0.9519 - val_loss: 0.1937 - val_accuracy: 0.9200
Epoch 3/10
200/200 [==============================] - 5s 24ms/step - loss: 0.0915 - accuracy: 0.9691 - val_loss: 0.2237 - val_accuracy: 0.9176
Epoch 4/10
200/200 [==============================] - 5s 26ms/step - loss: 0.0643 - accuracy: 0.9786 - val_loss: 0.2633 - val_accuracy: 0.9152
Epoch 5/10
200/200 [==============================] - 7s 36ms/step - loss: 0.0482 - accuracy: 0.9851 - val_loss: 0.3496 - val_accuracy: 0.9076
Epoch 6/10
200/200 [==============================] - 7s 33ms/step - loss: 0.0335 - accuracy: 0.9902 - val_loss: 0.3346 - val_accuracy: 0.9136
Epoch 7/10
200/200 [==============================] - 6s 31ms/step - loss: 0.0307 - accuracy: 0.9916 - val_loss: 0.3767 - val_accuracy: 0.9105

In [42]:
y_pred = model1.predict_classes(X_test)

In [43]:
from sklearn.metrics import confusion_matrix

In [44]:
confusion_matrix(y_test,y_pred)

array([[2822,  285],
       [ 200, 2179]], dtype=int64)

In [45]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test , y_pred)

0.9115931461903026